In [1]:
import os 
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold, content_types
from collections.abc import Iterable
from function_calls import emotionality_analyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
api_key = os.getenv("API_KEY")
genai.configure(api_key=api_key)

In [3]:
tools = {
  "name": "emotionality_analyzer",
  "description": "Analyzes the emotionality and exaggeration level in a given text and returns a scaled score.",
  "parameters": {
    "type": "object",
    "properties": {
      "text": {
        "type": "string",
        "description": "The text content to analyze for emotional intensity and exaggeration."
      }
    },
    "required": ["text"]
  }
}

In [4]:
generation_config = {
    "max_output_tokens": 4000, # less output, means faster
    "response_mime_type": "text/plain",
    "temperature": 1, # higher temp --> more risks the model takes with choices
    "top_p": 0.95, # how many tokens are considered when producing outputs
    "top_k": 40, # token is selected from 40 likely tokens
}

safety_settings = {
  HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

functions = [emotionality_analyzer]

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-002",
    generation_config=generation_config,
    safety_settings=safety_settings,
    tools = functions,
    system_instruction="You are trying to fight against misinformation by scoring different articles on their factuality factors. In your responses, do not use copyrighted material and be concise. Do not assess an article until you are given a factuality factor to grade on.",
)

In [5]:
tool_config = {
    "function_calling_config": {
        "mode": "AUTO", 
        "allowed_function_names": ["emotionality_analyzer"]
    },
}

In [6]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [8]:
response = chat.send_message("Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!. \
Given the previous sentence, rate the sentence based on the defined microfactors of sensationalism from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling: \
1. Sensationalism Detection: Identify instances of sensationalism in titles and main content. \
2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
3. Linguistic Database Comparison: Match linguistic features against databases of both \
trusted and untrusted sources to ascertain reliability.\
Then, combine them into an overall score for sensationalism.\
Lastly please phrase overall score as 'Normal Prompting Overall sensationalism: {score}'\
where score is a float and the phrase should be in plain text with no bolding or italics")
response.text

emotionalaity_score: 4.3


'1. **Sensationalism Detection (9/10):** The title uses strong emotional language ("Shocking," "Secret," "Change Humanity Forever") which are common sensationalist tactics.  The claim is also very broad and lacks specifics.\n\n2. **Emotion Analysis (10/10):**  The title is designed to evoke strong emotions (shock, excitement, anticipation) without providing any factual basis. The exaggeration is evident in the claim of changing humanity forever.\n\n3. **Linguistic Database Comparison (Not Applicable/10):** I lack the functionality to compare the text against databases of reliable/unreliable sources.\n\nNormal Prompting Overall sensationalism: 4.3\n'

In [16]:
# tool_config = tool_config_from_mode("auto")

response = model.generate_content("Given the inputted article rate it based on sensationalism from a score of 1 to 10.\
First, rate each chunk of the article based on the defined microfactors of sensationalism: \
1. Sensationalism Detection: Identify instances of sensationalism in titles and main content. \
2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
3. Linguistic Database Comparison: Match linguistic features against databases of both \
trusted and untrusted sources to ascertain reliability.\
This should be done paragraph by paragraph. \
Then, combine them into an overall score for sensationalism.\
Lastly please phrase overall score as 'Normal Prompting Overall sensationalism: {score}'\
where score is a float and the phrase should be in plain text with no bolding or italics", stream=True, tools=tools, tool_config=tool_config)
for chunk in response:
    text_chunk = chunk.text
    print(chunk.text)
    

KeyError: 'object'